In [22]:
import pandas as pd 
import glob

list_dfs = []
for f in glob.glob(r'C:/Users/15516/Desktop/数据处理/导出数据/*.xls'):
    print('合并列表', f)
    f1 = pd.read_excel(f)
    list_dfs.append(f1)

df3 = pd.concat(list_dfs, axis=0, ignore_index=True)
df3 = df3[['标题名称', '手机', '座机', '地址', '城市']]
df3['企业名称'] = df3['标题名称'].replace('地址', '', regex=True)

# 删除企业名称中不含有"口腔"、"齿科"或"牙科"的行
keywords = ['口腔', '齿科', '牙科']
df3 = df3[df3['企业名称'].str.contains('|'.join(keywords))]

# 重置索引
df3.reset_index(drop=True, inplace=True)

# 打印结果
df3 = df3[['企业名称','城市']]



df = pd.read_excel(r'C:/Users/15516/Desktop/市场新客0718.xlsx')[['企业名称','所属城市','所属区县']]
df['城市'] = df['所属城市'] + df['所属区县']
df = df[['企业名称','城市']]


merged_df = pd.merge(df, df3, on='城市', how='outer')

合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_090947.350.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091044.390.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091144.524.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091237.661.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091335.602.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091432.806.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091524.411.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091611.127.xls
合并列表 C:/Users/15516/Desktop/数据处理/导出数据\2023-07-18_091627.831.xls


In [23]:
import pandas as pd
from ngram import NGram
import re

def extract_prefix(name):
    # 使用正则表达式提取"口腔"、"齿科"或"牙科"之前的部分
    match = re.search(r'^(.*?)(口腔|齿科|牙科)', name)
    if match:
        return match.group(1)
    return name

def calculate_similarity(name1, name2):
    if not name1 or not name2:
        return 0.0

    ngram = NGram(N=2)  # 指定 N=2 用于计算 bigram 相似度
    similarity = ngram.compare(name1, name2)
    return similarity

def match_companies(row):
    name1 = str(row['企业名称_x'])
    name2 = str(row['企业名称_y'])

    name1_prefix = extract_prefix(name1)
    name2_prefix = extract_prefix(name2)

    similarity = calculate_similarity(name1_prefix, name2_prefix)
    return similarity

# 示例数据，假设df是一个DataFrame，含有'企业名称_x'和'企业名称_y'两列
df = merged_df

# 将'企业名称_x'和'企业名称_y'转换为字符串
df['企业名称_x'] = df['企业名称_x'].astype(str)
df['企业名称_y'] = df['企业名称_y'].astype(str)

# 使用apply函数计算相似度
df['相似度'] = df.apply(match_companies, axis=1)

# 保留相似度大于0.75的行
df = df.query('相似度 > 0.75')

# 使用drop_duplicates()方法去重，保留第一个出现的重复行
df = df.drop_duplicates()

# 输出结果
print(df)


                           企业名称_x      城市                 企业名称_y  相似度
55387            大连金普新区怡康口腔诊所有限公司  大连市金州区             大连金普新区怡康口腔  1.0
59391               苏州艾齿嘉口腔集团有限公司  苏州市姑苏区                苏州艾齿嘉口腔  1.0
60069               苏州艾齿嘉口腔管理有限公司  苏州市姑苏区                苏州艾齿嘉口腔  1.0
66922               太原市小店区成晓丽口腔诊所  太原市小店区          太原市小店区成晓丽口腔诊所  1.0
69342                   昌乐闫会娟口腔诊所  潍坊市昌乐县              昌乐闫会娟口腔诊所  1.0
71818   重庆豪品口腔诊所管理连锁有限公司渝北宝桐路口腔诊所  重庆市渝北区    重庆豪品口腔连锁(渝北飞湖路口腔诊所)  1.0
93081                永康淑扬口腔诊所有限公司  金华市永康市               永康淑扬口腔诊所  1.0
100613       威海华怡口腔门诊部有限公司环翠口腔门诊部  威海市环翠区           威海华怡口腔(环翠院区)  1.0
105827               天津和平卓奕口腔有限公司  天津市和平区            天津和平卓奕口腔门诊部  1.0
105862               天津和平卓奕口腔有限公司  天津市和平区            天津和平卓奕口腔门珍部  1.0
106646               南宁昆仑口腔诊所有限公司  南宁市兴宁区                 南宁昆仑口腔  1.0
106834               南宁昆仑口腔门诊有限公司  南宁市兴宁区                 南宁昆仑口腔  1.0
114837   山西津诚口腔医疗服务有限公司朔城区段双义口腔诊所  朔州市朔城区         山西津诚口腔医疗服务有限公司  1.0
141495              

In [24]:
df.to_excel('C:/Users/15516/Desktop/新客匹配.xlsx',index=False)